<a href="https://colab.research.google.com/github/divypandya/Inverse-Reinforcement-Learning/blob/divypandya-patch-1/IRL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Linear Programming formulation** 
#### **Inverse Reinforcement Learning in Ng & Russell 2000 paper: Algorithms for Inverse Reinforcement Learning**
##### [Paper Link](https://ai.stanford.edu/~ang/papers/icml00-irl.pdf)

*   **Here we have implemented IRL in Finite State Space using Linear Programming Optization** 

*   **Given**: 
 * A finite state space $S$ of $N$ **states**. 
 * $A = \{a_1,...,a_k\}$ is set of $k$ actions.
 * $P_{sa}(.)$ are the state **transition probabilities** upon taking action $a$ in state $s$.
 * $\gamma \in [0, 1)$ is the **discount factor**.
 * $R \ \ : S \rightarrow {\rm I\!R}$ is the **reinforcement function**, bounded in absolute way by $R_{max}$

*  **Goal** : We wish to find the set of possible reward functions $R$ such that $\pi$ is an optimal policy in MDP $(S, A, \{P_{sa}\}, \gamma, R)$




#### **Optimization Problem**

Given optimal policy $\pi(s)\equiv a_1$ our aim is to:

>  $ max \ \ \sum^{N}_{i=1} min_{a \in \{a_2,...,a_k\}} \{\ (P_{a_1}(i) - P_a(i))\ (I - \gamma P_{a_1})^{-1}R\} - \lambda ||R||_1$

> $ \ \ s.t. \ \ (P_{a_1} - P_a)(I - \gamma P_{a_1})^{-1}R \ \succeq 0, \ \ \forall a \in A\ \backslash \ a_1 $ \\
$\ \ \ \ \ \ \ \ \ \ \  |R_i| \leq R_{max}, \ \ i = 1,...,N$

where $P_a$ is $N \times N$ transition matrix for action $a$ and $P_a(i)$ denotes the $i^{th}$ row of $P_a$.

<br>

---

<br>


##### Above optimization problem can be written as a set of matrix equations.

> $max \ \ \{ \sum^{N}_{i=1}\{m_i - \lambda u_i\}\}$ <br>
> $\ \ s.t. \ \ \  m_i = min_{a \in A\backslash a_1} \{(P_{a_1}(i) - P_{a}(i))(I - \gamma P_a)^{-1}R\}, \ \ i = 1,...,N $ <br>
> $ \ \ \ \ \ \ \ \ \ \ (P_{a_1} - P_a)(I - \gamma P_{a_1}) \succeq 0, \ \ \ \forall a \in A\backslash a_1, \ \ i = 1,..., N $ <br>
> $ \ \ \ \ \ \ \ \ \ \ -U \leq R \leq U$ <br>
> $ \ \ \ \ \ \ \ \ \ \ |R_i| \leq R_{max}$

<br>

---
<br>

##### Now let's vectorize the above optimization task

* First select two dummy vectors $M$ and $U$ of length $N$,

* Construct inequality constraints matrix $A$. Each row of $A$ specifies coefficients of a linear inequality. Size of $A$ is $[2N(K+1), 3N]$

* $A = $

   \begin{bmatrix}
    -(P_{a_1} - P_a)(I - \gamma P_{a_1})^{-1} & I & 0 \\
    -(P_{a_1} - P_a)(I - \gamma P_{a_1})^{-1} & 0 & 0 \\
    I & 0 & 0 \\
    -I & 0 & 0 \\
    I & 0 & -I \\
    -I & 0 & -I
   \end{bmatrix}
<br>
*  A vector $c$ of length $3N$ which contains cofficiants of linear objective function to be minimized, i.e. <br>
$c = $

\begin{bmatrix}
    0\\
    1\\
    -\lambda 1
\end{bmatrix}

<br>

*  Also a vector $x$, which is to be found, of length $3N$ as <br>

\begin{bmatrix}
    R\\
    M\\
    U
\end{bmatrix} 

<br>

* vector $b$, the inequality contraint vector. Each element represents an upper bound on the corresponding value of $Ax$. \\
Size of b = $[2NK + 2N]$ <br>

 $b = $

\begin{bmatrix}
    0\\
    0\\
    R_{max}\\
    R_{max}\\
    0\\
    0
\end{bmatrix} 


<br>

---

<br>

##### Therefore now our new modified well vectorized optimized task is as follows: 

\begin{equation}
    argmax_x \ \ c^{T}x \\ 
    \ \ s.t. \ \ \  Ax \leq b
\end{equation}

Below is the python code for same.


In [0]:
import numpy as np
import scipy.optimize as opt

In [0]:
def normalize(val):
    min_val = np.min(val)
    max_val = np.max(val)

    return (val - min_val) / (max_val - min_val)

def linear_prog_irl(trans_probs, policy, gamma=0.1, l1=0.5, R_max=10):
    '''
    inputs:
        trans_probs       NxNx(N_ACTIONS) transition matrix, where N is #states
        policy            policy vector / map
        R_max             maximum possible value of recoverred rewards
        gamma             RL discount factor
        l1                l1 regularization parameter lambda

    returns:
        rewards           Nx1 reward vector

    '''
    
    N_s, _, N_a = np.shape(trans_probs)
    N_s = int(N_s)
    N_a = int(N_a)

    # Formulate a linear IRL problem
    A = np.zeros([2 * N_s * (N_a + 1), 3 * N_s])
    b = np.zeros([2 * N_s * (N_a + 1)])
    c = np.zeros([3 * N_s])

    for i in range(N_s):
        a_opt = int(policy[i])
        tmp_inv = np.linalg.inv(np.identity(N_s) - gamma * trans_probs[:, :, a_opt])

        cnt = 0
        for a in range(N_a):
            if a != a_opt:
                A[i * (N_a - 1) + cnt, :N_s] = - \
                    np.dot(trans_probs[i, :, a_opt] - trans_probs[i, :, a], tmp_inv)
                A[N_s * (N_a - 1) + i * (N_a - 1) + cnt, :N_s] = - \
                    np.dot(trans_probs[i, :, a_opt] - trans_probs[i, :, a], tmp_inv)
                A[N_s * (N_a - 1) + i * (N_a - 1) + cnt, N_s + i] = 1
                cnt += 1

    for i in range(N_s):
        A[2 * N_s * (N_a - 1) + i, i] = 1
        b[2 * N_s * (N_a - 1) + i] = R_max

    for i in range(N_s):
        A[2 * N_s * (N_a - 1) + N_s + i, i] = -1
        b[2 * N_s * (N_a - 1) + N_s + i] = 0

    for i in range(N_s):
        A[2 * N_s * (N_a - 1) + 2 * N_s + i, i] = 1
        A[2 * N_s * (N_a - 1) + 2 * N_s + i, 2 * N_s + i] = -1

    for i in range(N_s):
        A[2 * N_s * (N_a - 1) + 3 * N_s + i, i] = 1
        A[2 * N_s * (N_a - 1) + 3 * N_s + i, 2 * N_s + i] = -1

    for i in range(N_s):
        c[N_s:2 * N_s] = -1
        c[2 * N_s:] = l1

    res = opt.linprog(c, A_ub=A, b_ub=b)
    reward = res.x[:N_s]
    reward = normalize(reward) * R_max

    return reward


# **Results**
### Ground Truth Reward Function
<div>
<img src= "https://drive.google.com/uc?id=1E2DrWHR4xAJZ2gSkSr6NPKMnfip1keuU" width="300" height="300"/> &emsp; &emsp; &emsp; &emsp;
<img src= "https://drive.google.com/uc?id=1-HRmYDX2SaSvhvCmPbwF2C7XunUH_mdA" width="300" height='300'/>
</div>

<br>

### Recovered Reward Function
<div>
<img src= "https://drive.google.com/uc?id=1-8dEOXwpzdxax5PkZS64hVyr4FWYhU3J" width="300" height="300"/> &emsp; &emsp; &emsp; &emsp;
<img src= "https://drive.google.com/uc?id=1-9hjln_yqCV9QOcDIjyxbVsWwc9Kp_Q2" width="300" height='300'/>
</div>


